In [4]:
import pandas as pd
import numpy as np

# Load the CSV file
file_path = r'..\data\raw\retail_data.csv'
df = pd.read_csv(file_path)

In [5]:
# Print the number of rows before cleaning
print("Number of rows before cleaning:", len(df))

# Print the number of missing values before cleaning for specified columns
columns_to_check = ['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Age', 'Gender', 'Income', 'Customer_Segment', 'Date', 'Time', 'Product_Category', 'Product_Brand', 'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method', 'Order_Status', 'Ratings', 'products']
print("Number of missing values before cleaning:")
print(df[columns_to_check].isnull().sum())

Number of rows before cleaning: 302010
Number of missing values before cleaning:
Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Time                350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64


In [6]:
# Step 1: Drop rows where any of the specified columns have missing values
df_cleaned = df.dropna(subset=columns_to_check)

# Print the number of rows after cleaning
print("Number of rows after cleaning:", len(df_cleaned))

# Print the number of missing values after cleaning for specified columns
print("Number of missing values after cleaning:")
print(df_cleaned[columns_to_check].isnull().sum())

Number of rows after cleaning: 296994
Number of missing values after cleaning:
Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Time                0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
dtype: int64


In [7]:
# Step 2: Check for duplicates based on both Transaction_ID and Customer_ID
duplicates = df_cleaned[df_cleaned.duplicated(subset=['Transaction_ID', 'Customer_ID'], keep=False)]

# Print the first 3 sets of duplicates side by side
if not duplicates.empty:
    print("First 3 duplicate sets based on Transaction_ID and Customer_ID:\n")
    grouped_duplicates = duplicates.groupby(['Transaction_ID', 'Customer_ID'])
    
    for name, group in list(grouped_duplicates)[:3]:
        print(f"Duplicate group for Transaction_ID = {name[0]} and Customer_ID = {name[1]}:\n")
        print(group)
        print("\n" + "="*80 + "\n")

    # Remove duplicates, keeping only the first occurrence
    df_cleaned = df_cleaned.drop_duplicates(subset=['Transaction_ID', 'Customer_ID'], keep='first')

    # Print confirmation
    print(f"Removed {len(duplicates)} duplicates. New DataFrame length: {len(df_cleaned)}")

First 3 duplicate sets based on Transaction_ID and Customer_ID:

Duplicate group for Transaction_ID = 1005039.0 and Customer_ID = 84239.0:

        Transaction_ID  Customer_ID       Name               Email  \
165944       1005039.0      84239.0  John Tate  Andrea27@gmail.com   
301783       1005039.0      84239.0  John Tate  Andrea27@gmail.com   

               Phone              Address  City   State  Zipcode  Country  \
165944  9.064051e+09  7563 Pittman Tunnel  Bonn  Berlin  35327.0  Germany   
301783  9.064051e+09  7563 Pittman Tunnel  Bonn  Berlin  35327.0  Germany   

        ...  Total_Amount Product_Category Product_Brand Product_Type  \
165944  ...   1114.320116          Grocery     Coca-Cola        Water   
301783  ...   1114.320116          Grocery     Coca-Cola        Water   

       Feedback  Shipping_Method Payment_Method Order_Status  Ratings  \
165944      Bad         Same-Day     Debit Card      Pending      1.0   
301783      Bad         Same-Day           Cash    

In [8]:
# Step 3: Handle missing 'Amount', 'Total_Purchases', and 'Total_Amount' values
columns_to_check_amounts = ['Amount', 'Total_Purchases', 'Total_Amount']
print("Number of missing values before handling 'Amount', 'Total_Purchases', and 'Total_Amount':")
print(df_cleaned[columns_to_check_amounts].isnull().sum())

# Fill in missing 'Amount' values
print("Filling in missing 'Amount' values...")
df_cleaned['Amount'] = df_cleaned.apply(lambda row: row['Total_Amount'] / row['Total_Purchases'] if pd.isnull(row['Amount']) and pd.notnull(row['Total_Amount']) and pd.notnull(row['Total_Purchases']) else row['Amount'], axis=1)
print("Missing values after filling 'Amount':")
print(df_cleaned['Amount'].isnull().sum())

# Fill in missing 'Total_Purchases' values
print("Filling in missing 'Total_Purchases' values...")
df_cleaned['Total_Purchases'] = df_cleaned.apply(lambda row: np.round(row['Total_Amount'] / row['Amount']) if pd.isnull(row['Total_Purchases']) and pd.notnull(row['Total_Amount']) and pd.notnull(row['Amount']) else row['Total_Purchases'], axis=1)
print("Missing values after filling 'Total_Purchases':")
print(df_cleaned['Total_Purchases'].isnull().sum())

# Fill in missing 'Total_Amount' values
print("Filling in missing 'Total_Amount' values...")
df_cleaned['Total_Amount'] = df_cleaned.apply(lambda row: row['Total_Purchases'] * row['Amount'] if pd.isnull(row['Total_Amount']) and pd.notnull(row['Total_Purchases']) and pd.notnull(row['Amount']) else row['Total_Amount'], axis=1)
print("Missing values after filling 'Total_Amount':")
print(df_cleaned['Total_Amount'].isnull().sum())

Number of missing values before handling 'Amount', 'Total_Purchases', and 'Total_Amount':
Amount             349
Total_Purchases    355
Total_Amount       342
dtype: int64
Filling in missing 'Amount' values...
Missing values after filling 'Amount':
1
Filling in missing 'Total_Purchases' values...
Missing values after filling 'Total_Purchases':
0
Filling in missing 'Total_Amount' values...
Missing values after filling 'Total_Amount':
1


In [9]:
# Step 4: Drop rows with any missing values in 'Amount', 'Total_Purchases', or 'Total_Amount'
df_cleaned = df_cleaned.dropna(subset=columns_to_check_amounts)

# Function to parse dates with multiple formats
def parse_dates(date_str):
    try:
        # Try parsing with the first format MM/DD/YYYY
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    except ValueError:
        try:
            # Try parsing with the second format MM-DD-YY
            return pd.to_datetime(date_str, format='%m-%d-%y')
        except ValueError:
            return pd.NaT

In [10]:
# Step 5: Remove 'Year' and 'Month' columns
df_cleaned = df_cleaned.drop(columns=['Year', 'Month'])

# Print the first few rows of the Date column before conversion
print("First 10 rows of the 'Date' column before conversion:")
print(df_cleaned['Date'].head(10))

# Apply the custom date parsing function
df_cleaned['Date'] = df_cleaned['Date'].apply(parse_dates)

# Verify the 'Date' column after conversion
print("First 10 rows of the 'Date' column after conversion:")
print(df_cleaned['Date'].head(10))

# Delete rows where 'Date' is NaT
df_cleaned = df_cleaned.dropna(subset=['Date'])

# Print the total number of rows after deletion
print("Total number of rows after deleting NaT dates:", len(df_cleaned))

# Step 6: Create 'Year', 'Month', and 'Day_of_Week' columns
df_cleaned['Year'] = df_cleaned['Date'].dt.year
df_cleaned['Month'] = df_cleaned['Date'].dt.strftime('%B')  # Full month name (e.g., 'September')
df_cleaned['Day_of_Week'] = df_cleaned['Date'].dt.strftime('%A')  # Full day name (e.g., 'Monday')

# Verify the new columns
print("First 10 rows of the new 'Year', 'Month', and 'Day_of_Week' columns:")
print(df_cleaned[['Year', 'Month', 'Day_of_Week']].head(10))

First 10 rows of the 'Date' column before conversion:
0     9/18/2023
1    12/31/2023
2     4/26/2023
3      05-08-23
4      01-10-24
5     9/21/2023
6     6/26/2023
7     3/24/2023
8      01-06-24
9      10-04-23
Name: Date, dtype: object
First 10 rows of the 'Date' column after conversion:
0   2023-09-18
1   2023-12-31
2   2023-04-26
3   2023-05-08
4   2024-01-10
5   2023-09-21
6   2023-06-26
7   2023-03-24
8   2024-01-06
9   2023-10-04
Name: Date, dtype: datetime64[ns]
Total number of rows after deleting NaT dates: 294679
First 10 rows of the new 'Year', 'Month', and 'Day_of_Week' columns:
   Year      Month Day_of_Week
0  2023  September      Monday
1  2023   December      Sunday
2  2023      April   Wednesday
3  2023        May      Monday
4  2024    January   Wednesday
5  2023  September    Thursday
6  2023       June      Monday
7  2023      March      Friday
8  2024    January    Saturday
9  2023    October   Wednesday


In [11]:
# Step 7: Drop rows where either 'Zipcode' or 'Address' is missing
df_cleaned = df_cleaned.dropna(subset=['Zipcode', 'Address'])

# Step 8: Reset index to avoid any potential issues
df_cleaned = df_cleaned.reset_index(drop=True)

# Define a function to fill missing values based on other columns
def fill_missing_values(group):
    for col in ['City', 'State', 'Country']:
        if pd.isnull(group[col].iloc[0]):
            non_null_value = group[col].dropna().unique()
            if len(non_null_value) == 1:
                group[col].fillna(non_null_value[0], inplace=True)
    return group

# Step 9: Group by 'City', 'State', and 'Country' and apply the fill_missing_values function
df_cleaned = df_cleaned.groupby(['City', 'State', 'Country'], as_index=False).apply(fill_missing_values)

# Step 10: Drop any remaining rows with missing 'City', 'State', or 'Country' values after filling
df_cleaned = df_cleaned.dropna(subset=['City', 'State', 'Country'])

In [12]:
# Final check for any remaining missing values in the entire DataFrame
print("Final check for any remaining missing values in all columns:")
print(df_cleaned.isnull().sum())

# Print the total number of rows after all cleaning steps
print("Total number of rows after cleaning and processing:", len(df_cleaned))

#Save
csv_output_path = r'..\data\cleaned\retail_data_cleaned.csv'
df_cleaned.to_csv(csv_output_path, index=False)
print(f"Data successfully saved as CSV to: {csv_output_path}")

Final check for any remaining missing values in all columns:
Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Time                0
Total_Purchases     0
Amount              0
Total_Amount        0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
Year                0
Month               0
Day_of_Week         0
dtype: int64
Total number of rows after cleaning and processing: 293267
Data successfully saved as CSV to: ..\data\cleaned\retail_data_cleaned.csv
